In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats


from sklearn.linear_model import Ridge,Lasso
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings('ignore')

In [2]:
df_train=pd.read_csv('data/train.csv')
df_test=pd.read_csv('data/test.csv') 
df_test['SalePrice'] = 0

In [3]:
str_list = ['Alley','Heating','HeatingQC','CentralAir','Electrical','BsmtQual','BsmtCond','SaleType','SaleCondition',\
           'PavedDrive']
#str_list = []
num_cat = ['Fireplaces', 'TotRmsAbvGrd', 'FullBath', 'GarageCars', 'OverallQual']
#num_cat = []
num_list = ['MasVnrArea', 'GarageYrBlt', 'YearRemodAdd', 'YearBuilt', '1stFlrSF', 'TotalBsmtSF', 'GarageArea', 'GrLivArea']
all_list = ['Id'] + str_list + num_cat + num_list + ['SalePrice']
df_total = pd.concat([df_train,df_test],axis=0)

#print(df_total.shape)
df_use = df_total[all_list]
#print(df_use.shape)
#print(df_use.tail())
#print(df_train[str_list])
# str_list = ['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', \
#             'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', \
#             'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', \
#             'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', \
#             'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', \
#             'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', \
#             'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']


In [4]:
from sklearn.base import TransformerMixin
class DataFrameImputer(TransformerMixin):

    def __init__(self):
        """Impute missing values.

        Columns of dtype object are imputed with the most frequent value 
        in column.

        Columns of other types are imputed with mean of column.

        """
    def fit(self, X, y=None):

        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].mean() for c in X],
            index=X.columns)

        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill)


# fill nan

In [5]:
df_remove_nan = DataFrameImputer().fit_transform(df_use)
print(df_remove_nan.isnull().sum().max())

0


In [6]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils import check_array
from sklearn.preprocessing import LabelEncoder
from scipy import sparse

class CategoricalEncoder(BaseEstimator, TransformerMixin):
    

    def __init__(self, encoding='onehot', categories='auto', dtype=np.float64,
                 handle_unknown='error'):
        self.encoding = encoding
        self.categories = categories
        self.dtype = dtype
        self.handle_unknown = handle_unknown

    def fit(self, X, y=None):

        if self.encoding not in ['onehot', 'onehot-dense', 'ordinal']:
            template = ("encoding should be either 'onehot', 'onehot-dense' "
                        "or 'ordinal', got %s")
            raise ValueError(template % self.handle_unknown)

        if self.handle_unknown not in ['error', 'ignore']:
            template = ("handle_unknown should be either 'error' or "
                        "'ignore', got %s")
            raise ValueError(template % self.handle_unknown)

        if self.encoding == 'ordinal' and self.handle_unknown == 'ignore':
            raise ValueError("handle_unknown='ignore' is not supported for"
                             " encoding='ordinal'")

        X = check_array(X, dtype=np.object, accept_sparse='csc', copy=True)
        n_samples, n_features = X.shape

        self._label_encoders_ = [LabelEncoder() for _ in range(n_features)]

        for i in range(n_features):
            le = self._label_encoders_[i]
            Xi = X[:, i]
            if self.categories == 'auto':
                le.fit(Xi)
            else:
                valid_mask = np.in1d(Xi, self.categories[i])
                if not np.all(valid_mask):
                    if self.handle_unknown == 'error':
                        diff = np.unique(Xi[~valid_mask])
                        msg = ("Found unknown categories {0} in column {1}"
                               " during fit".format(diff, i))
                        raise ValueError(msg)
                le.classes_ = np.array(np.sort(self.categories[i]))

        self.categories_ = [le.classes_ for le in self._label_encoders_]

        return self

    def transform(self, X):
        X = check_array(X, accept_sparse='csc', dtype=np.object, copy=True)
        n_samples, n_features = X.shape
        X_int = np.zeros_like(X, dtype=np.int)
        X_mask = np.ones_like(X, dtype=np.bool)

        for i in range(n_features):
            valid_mask = np.in1d(X[:, i], self.categories_[i])

            if not np.all(valid_mask):
                if self.handle_unknown == 'error':
                    diff = np.unique(X[~valid_mask, i])
                    msg = ("Found unknown categories {0} in column {1}"
                           " during transform".format(diff, i))
                    raise ValueError(msg)
                else:
                    X_mask[:, i] = valid_mask
                    X[:, i][~valid_mask] = self.categories_[i][0]
            X_int[:, i] = self._label_encoders_[i].transform(X[:, i])

        if self.encoding == 'ordinal':
            return X_int.astype(self.dtype, copy=False)

        mask = X_mask.ravel()
        n_values = [cats.shape[0] for cats in self.categories_]
        n_values = np.array([0] + n_values)
        indices = np.cumsum(n_values)

        column_indices = (X_int + indices[:-1]).ravel()[mask]
        row_indices = np.repeat(np.arange(n_samples, dtype=np.int32),
                                n_features)[mask]
        data = np.ones(n_samples * n_features)[mask]

        out = sparse.csc_matrix((data, (row_indices, column_indices)),
                                shape=(n_samples, indices[-1]),
                                dtype=self.dtype).tocsr()
        if self.encoding == 'onehot-dense':
            return out.toarray()
        else:
            return out

In [7]:
from sklearn.base import BaseEstimator,TransformerMixin
#select columns and transit to array

class DataFrameSelector(BaseEstimator,TransformerMixin):
    def __init__(self,feature_names):
        self.feature_names = feature_names
    def fit(self,X,y=None):
        return self
    def transform(self,X):
        return X[self.feature_names].values

In [8]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Imputer   
from sklearn.pipeline import FeatureUnion
#CategoricalEncoder(encoding='onehot-dense')

num_attribs = num_list
num_pipeline = Pipeline([
               ('selector',DataFrameSelector(num_attribs)),      
#               ('std_scaler',StandardScaler()),
              ])
const_pipeline = Pipeline([
               ('selector',DataFrameSelector(['Id','SalePrice'])),      
              ])
num_cat_attribs = str_list + num_cat
num_obj_pipeline = Pipeline([
                  ('selector',DataFrameSelector(num_cat_attribs)),
                  ('cat_encoder',CategoricalEncoder(encoding='onehot-dense')),
              ])

In [9]:
full_pipeline = FeatureUnion(transformer_list=
                             [
                              ('const_pipeline',const_pipeline),
                              ('num_pipeline',num_pipeline),
                              
                              ('num_obj_pipeline',num_obj_pipeline)
                             ],
                             )

In [10]:
print(df_remove_nan.head())
df_final = full_pipeline.fit_transform(df_remove_nan)


   Id Alley Heating HeatingQC CentralAir Electrical BsmtQual BsmtCond  \
0   1  Grvl    GasA        Ex          Y      SBrkr       Gd       TA   
1   2  Grvl    GasA        Ex          Y      SBrkr       Gd       TA   
2   3  Grvl    GasA        Ex          Y      SBrkr       Gd       TA   
3   4  Grvl    GasA        Gd          Y      SBrkr       TA       Gd   
4   5  Grvl    GasA        Ex          Y      SBrkr       Gd       TA   

  SaleType SaleCondition    ...     OverallQual  MasVnrArea  GarageYrBlt  \
0       WD        Normal    ...               7       196.0       2003.0   
1       WD        Normal    ...               6         0.0       1976.0   
2       WD        Normal    ...               7       162.0       2001.0   
3       WD       Abnorml    ...               7         0.0       1998.0   
4       WD        Normal    ...               8       350.0       2000.0   

   YearRemodAdd  YearBuilt  1stFlrSF  TotalBsmtSF  GarageArea  GrLivArea  \
0          2003       2003  

In [11]:
df_final.shape

(2919, 97)

In [12]:
df_final = pd.DataFrame(df_final)


In [13]:
df_final.tail(20)

,0,1,2,3,4,5,6,7,8,9,...,87,88,89,90,91,92,93,94,95,96
2899,2900.0,0.0,257.0,1975.000000,1975.0,1975.0,1210.0,1128.0,528.0,1210.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2900,2901.0,0.0,0.0,1958.000000,1958.0,1958.0,1650.0,1632.0,518.0,1650.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2901,2902.0,0.0,0.0,2000.000000,2000.0,2000.0,1403.0,1381.0,470.0,1403.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2902,2903.0,0.0,198.0,2005.000000,2006.0,2005.0,1960.0,1728.0,714.0,1960.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2903,2904.0,0.0,382.0,2005.000000,2006.0,2005.0,1838.0,1838.0,682.0,1838.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2904,2905.0,0.0,0.0,1951.000000,1951.0,1951.0,1600.0,0.0,270.0,1600.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2905,2906.0,0.0,200.0,1997.000000,1997.0,1997.0,1368.0,1288.0,784.0,1368.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2906,2907.0,0.0,0.0,1977.000000,1977.0,1977.0,616.0,264.0,336.0,1304.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2907,2908.0,0.0,0.0,1968.000000,2003.0,1968.0,874.0,864.0,288.0,874.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2908,2909.0,0.0,0.0,1970.000000,1970.0,1970.0,1652.0,1652.0,928.0,1652.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [14]:
df_final_train = df_final[df_final[1]>0]
df_final_test = df_final[df_final[1]<0.1]

Save2File=pd.DataFrame()
Save2File['Id']=df_final_test[0]
print(Save2File.head())

df_final_test = df_final_test.drop(df_final_test.columns[0:2],axis=1)
print(df_final_test.head())

print(df_final.shape)
print(df_final_train.shape)
print(df_final_test.shape)
X_train_features = df_final_train.drop(df_final_train.columns[0:2],axis=1)
print('df_final_test.shape',df_final_test.shape,'X_train_features.shape',X_train_features.shape)
#y_train_features = df_final_train[1]
y_train_features = np.log(df_final_train[1])
print(y_train_features[0:5])

          Id
1460  1461.0
1461  1462.0
1462  1463.0
1463  1464.0
1464  1465.0
         2       3       4       5       6       7      8       9    10   11  \
1460    0.0  1961.0  1961.0  1961.0   896.0   882.0  730.0   896.0  1.0  0.0   
1461  108.0  1958.0  1958.0  1958.0  1329.0  1329.0  312.0  1329.0  1.0  0.0   
1462    0.0  1997.0  1998.0  1997.0   928.0   928.0  482.0  1629.0  1.0  0.0   
1463   20.0  1998.0  1998.0  1998.0   926.0   926.0  470.0  1604.0  1.0  0.0   
1464    0.0  1992.0  1992.0  1992.0  1280.0  1280.0  506.0  1280.0  1.0  0.0   

     ...    87   88   89   90   91   92   93   94   95   96  
1460 ...   0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  
1461 ...   0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  
1462 ...   0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  
1463 ...   0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  
1464 ...   0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  

[5 rows x 95 columns]
(2919, 97)
(1460, 97)
(1459, 95)
df_final_test

In [15]:
X_train,X_test,y_train,y_test=train_test_split(X_train_features,y_train_features,test_size=0.33)
print(X_train.shape)

(978, 95)


In [16]:
#degree=2
#alpha=0.1
from sklearn.metrics import mean_squared_error




print(df_final_train.shape)
print(df_final_test.shape)
score_best =1
score_train=0
degree_best=0
alpha_best =0
for alpha in np.arange(0.2,1.0,0.2):
    for degree in range(1,3):
        for epoch in range(1,11):
            Poly=PolynomialFeatures(degree=degree)
            #df_sample=df_train_features[features_all_corr05].sample(frac=1).reset_index(drop=True)
            X_train,X_test,y_train,y_test=train_test_split(X_train_features,y_train_features,test_size=0.33)
            X_train_transform =Poly.fit_transform(X_train)
            X_test_transform  =Poly.fit_transform(X_test) 
            
            X_final_test  = Poly.fit_transform(df_final_test) # real testdata
            
            print(type(X_train),type(X_test_transform))
            clf=linear_model.Lasso(alpha=alpha)
            clf.fit(X_train_transform,y_train)
            y_train_pred=clf.predict(X_train_transform)
            y_test_pred=clf.predict(X_test_transform) 
            #print('X_test.shape',X_test.shape,'X_test_transform.shape',X_test_transform.shape,'y_test_pred.shape',y_test_pred.shape)
            print('epoch={:d},alpha={:.2f},degree={:d}, train data: r2_score={:.2f}'
                  .format(epoch,alpha,degree,r2_score(y_train_pred,y_train)))
            print('epoch={:d},alpha={:.2f},degree={:d}, test data: r2_score={:.2f}'
                  .format(epoch,alpha,degree,r2_score(y_test_pred,y_test)))
            lin_mse = mean_squared_error(y_test,y_test_pred)
            lin_rmse = np.sqrt(lin_mse)
            print('Mean Square Error: ',lin_rmse)
            if(mean_squared_error(y_test_pred,y_test) < score_best):
                
                score_best  = mean_squared_error(y_test_pred,y_test)
                degree_best = degree
                alpha_best  = alpha
                score_train = mean_squared_error(y_train_pred,y_train)
                print('COME IN mean_squared_error',score_best,'degree best',degree_best,'alpha_best',alpha_best,'score_train',score_train)
                #print('df_final_test shape',df_final_test.shape,'X_test_transform shape',X_test_transform.shape)
                #print('y_train_pred shape',y_train_pred.shape,'y_train shape',len(y_train))
                
                #print(X_test_transform[:6])
                #print(df_final_test.head())
                y_test_final = clf.predict(X_final_test) # prediction for real test data
                Save2File['SalePrice']=y_test_final
print("The best case: epoch={:d},alpha={:.2f},degree={:d}, test_score={:.2f}, train_score={:.2f}"
      .format(epoch,alpha_best,degree_best,np.sqrt(score_best),np.sqrt(score_train)))





(1460, 97)
(1459, 95)
<class 'pandas.core.frame.DataFrame'> <class 'numpy.ndarray'>
epoch=1,alpha=0.20,degree=1, train data: r2_score=0.66
epoch=1,alpha=0.20,degree=1, test data: r2_score=0.78
Mean Square Error:  0.15711622003477826
COME IN mean_squared_error 0.024685506598016858 degree best 1 alpha_best 0.2 score_train 0.04183755293759379
<class 'pandas.core.frame.DataFrame'> <class 'numpy.ndarray'>
epoch=2,alpha=0.20,degree=1, train data: r2_score=0.67
epoch=2,alpha=0.20,degree=1, test data: r2_score=0.75
Mean Square Error:  0.16205508003130056
<class 'pandas.core.frame.DataFrame'> <class 'numpy.ndarray'>
epoch=3,alpha=0.20,degree=1, train data: r2_score=0.68
epoch=3,alpha=0.20,degree=1, test data: r2_score=0.65
Mean Square Error:  0.19588723093348237
<class 'pandas.core.frame.DataFrame'> <class 'numpy.ndarray'>
epoch=4,alpha=0.20,degree=1, train data: r2_score=0.79
epoch=4,alpha=0.20,degree=1, test data: r2_score=0.63
Mean Square Error:  0.253805837121665
<class 'pandas.core.frame.D

epoch=6,alpha=0.40,degree=2, train data: r2_score=0.89
epoch=6,alpha=0.40,degree=2, test data: r2_score=0.84
Mean Square Error:  0.15018695543961355
<class 'pandas.core.frame.DataFrame'> <class 'numpy.ndarray'>
epoch=7,alpha=0.40,degree=2, train data: r2_score=0.90
epoch=7,alpha=0.40,degree=2, test data: r2_score=0.82
Mean Square Error:  0.16150269425875022
<class 'pandas.core.frame.DataFrame'> <class 'numpy.ndarray'>
epoch=8,alpha=0.40,degree=2, train data: r2_score=0.89
epoch=8,alpha=0.40,degree=2, test data: r2_score=0.84
Mean Square Error:  0.1481502859505911
<class 'pandas.core.frame.DataFrame'> <class 'numpy.ndarray'>
epoch=9,alpha=0.40,degree=2, train data: r2_score=0.89
epoch=9,alpha=0.40,degree=2, test data: r2_score=0.84
Mean Square Error:  0.14886919008073818
<class 'pandas.core.frame.DataFrame'> <class 'numpy.ndarray'>
epoch=10,alpha=0.40,degree=2, train data: r2_score=0.89
epoch=10,alpha=0.40,degree=2, test data: r2_score=0.82
Mean Square Error:  0.1553557729719971
<class 

epoch=5,alpha=0.80,degree=2, train data: r2_score=0.87
epoch=5,alpha=0.80,degree=2, test data: r2_score=0.85
Mean Square Error:  0.14024615360864875
<class 'pandas.core.frame.DataFrame'> <class 'numpy.ndarray'>
epoch=6,alpha=0.80,degree=2, train data: r2_score=0.88
epoch=6,alpha=0.80,degree=2, test data: r2_score=0.85
Mean Square Error:  0.1462057928562298
<class 'pandas.core.frame.DataFrame'> <class 'numpy.ndarray'>
epoch=7,alpha=0.80,degree=2, train data: r2_score=0.87
epoch=7,alpha=0.80,degree=2, test data: r2_score=0.86
Mean Square Error:  0.13987847481060425
<class 'pandas.core.frame.DataFrame'> <class 'numpy.ndarray'>
epoch=8,alpha=0.80,degree=2, train data: r2_score=0.88
epoch=8,alpha=0.80,degree=2, test data: r2_score=0.84
Mean Square Error:  0.1492222298347164
<class 'pandas.core.frame.DataFrame'> <class 'numpy.ndarray'>
epoch=9,alpha=0.80,degree=2, train data: r2_score=0.89
epoch=9,alpha=0.80,degree=2, test data: r2_score=0.84
Mean Square Error:  0.14630366469542985
<class 'p